In [10]:
import geopandas as gpd
import polars as pl
from shapely import Point
import pandas as pd
pd.set_option('display.max_rows', 30)

In [2]:
comp_path = '../wca_export/WCA_export_Competitions.tsv'
res_path = '../wca_export/WCA_export_Results.tsv'

In [3]:
b_comps = (pl.read_csv(comp_path,sep='\t')).lazy().filter(pl.col('countryId')=='Brazil')
res = pl.read_csv(res_path,sep='\t').lazy()

b_res = b_comps.join(res,left_on='id',right_on='competitionId').select(["id","personId","personName",'latitude','longitude'])
b_res = b_res.groupby(['id','personId']).first().collect().to_pandas()
b_res['latitude'] /= 1000000
b_res['longitude'] /= 1000000

In [4]:
# Download the file here (note, I renamed the file extension): 
# https://github.com/giuliano-oliveira/geodata-br-states/blob/main/geojson/br_states.json
brazil = gpd.read_file('br_states.geojson') 

In [5]:
b_res['geometry'] = b_res.apply(lambda x: Point(x.longitude, x.latitude), axis=1)
geo_res = gpd.GeoDataFrame(b_res, crs=brazil.crs)

In [6]:
person_states = gpd.sjoin(brazil, geo_res, predicate='contains')

In [7]:
state_count = pd.DataFrame(person_states.groupby('personId')['Estado'].nunique().sort_values(ascending=False))

In [8]:
state_count.head(20)

,Estado
personId,
2007CINO01,20
2010GRAC02,20
2016SATO01,11
2015BELO02,11
2017MAND13,11
2015CARV06,10
2017NOVA05,9
2019MORA23,9
2015IACO01,9


In [1]:
# show the map, takes a bit of time
# person_states[person_states['personId'] == '2007CINO01'].explore()